### Energy Analysis Notebook
#### Micro Renewable Energy System - Generation, Tariffs, Export & Battery Optimisation

This notebook analyses **30 minute smart meter electricity data** to compare:
- Electricity tariffs
- Grid export vs battery storage strategies
- EV night charging
- Household usage behaviour

The goal is to determine the **most cost effective energy management strategy** for a modern home using:
- Solar PV
- Smart metering
- Time of use tariffs
- EV charging
- Battery storage


---

### Objectives

- Analyse real and synthetic **30 minute electricity consumption data**
- Compare multiple **tariff structures**

- Evaluate the financial impact of:
    - Exporting to the grid
    - Storing energy in a battery

- Compare two household profiles:
    - All day occupancy
    - Away 9am to 5pm with EV charging

- Determine:
    - Annual electricity cost
    - Export revenue
    - Battery payback period
    - ROI (Return on Investment)

    ---

### Case Study Definitions

#### Case Study 1 - All Day Occupancy
- High electricity usage throughout the day
- High solar self consumption


#### Case Study 2 - Away 9am to 5pm with EV Charging
- Low daytime usage
- High evening peak (5:30pm to 7:30pm)
- Overnight EV charging
- High solar generation during working hours

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)